# Words analysis

** Ojectif : ** Sara souhaite des fichiers de comptages de mots (genre, top 100 des mots les plus prononcés) par sexe et par genre de Films. Pas besoin de plot les wordclouds, juste de lui faire un tableau de comptage mot - fréquence.

- ## Imports

In [3]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import seaborn as sns

import os, sys, json, re, argparse, urllib2, html5lib
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup, Tag, UnicodeDammit
import pandas as pd
from pandas.io.json import json_normalize
import time
import math
from parse_script import *
from crawler import *

/home/duc-vinh/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/home/duc-vinh/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


- ### Script loader

In [4]:
def load_bygenre(genre):
    #load all dataframes of movies by genre, input is as tring, output a list of df
    genre=genre.lower()
    mypath = "../data/script/%s/clean/" %(genre)
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    output=[]
    for element in onlyfiles:
        output.append(pd.read_csv(mypath+element,sep='|',encoding='latin1',index_col=0))
    return output

def number_speech(df):
    m=len(df[df['gender']=='M'])
    f=len(df[df['gender']=='F'])
    return m,f

def nb_char_bygender(df):
    #print(set(df[df['gender']=='M']['character_clean']))
    m=len(set(df[df['gender']=='M']['character_clean']))
    f=len(set(df[df['gender']=='F']['character_clean']))
    return m,f

def normalize_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    normal = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore')
    return normal



In [5]:
def create_global_df(list_genre):
    #exemple
    liste = ['genre','nb_of_movies','nb_male','nb_female','nb_male_occurences','nb_female_occurences',
            'nb_prota_male','nb_prota_female']
    df_ = pd.DataFrame(columns=liste)
    for genre in list_genre:
        try:
            dfs = load_bygenre(genre)
            m=[]
            f=[]
            m_nb=[]
            f_nb=[]
            pro_m=[]
            pro_f=[]
            for df in dfs:
                #print(df.head(1))
                try:
                    m.append(number_speech(df)[0])
                    f.append(number_speech(df)[1])
                    m_nb.append(nb_char_bygender(df)[0])
                    f_nb.append(nb_char_bygender(df)[1])
                    pro_m.append(len(set(df[(df['principal_character']==True) & (df['gender']=='M') ]['character_clean'])))
                    pro_f.append(len(set(df[(df['principal_character']==True) & (df['gender']=='F') ]['character_clean'])))
                except Exception as e:
                    print(e)
            x = pd.DataFrame([genre,len(dfs),sum(m_nb),sum(f_nb),sum(m),sum(f),
                             sum(pro_m),sum(pro_f)],index=liste)
            x= x.transpose()
            df_ = df_.append(x,ignore_index=True)
            
        except Exception as e:
            print(e)
    return df_

In [6]:
#liste des genres
liste=[ u'Action',
 u'Adventure',
 u'Animation',
 u'Biography',
 u'Comedy',
 u'Crime',
 u'Drama',
 u'Family',
 u'Fantasy',
 u'History',
 u'Horror',
 u'Music',
 u'Musical',
 u'Mystery',
 u'Romance',
 u'Sci-Fi',
 u'Sport',
 u'Thriller',
 u'War',
 u'Western']

In [7]:
create_global_df(liste)

'principal_character'
'principal_character'
[Errno 2] No such file or directory: '../data/script/musical/clean/'
[Errno 2] No such file or directory: '../data/script/sport/clean/'
[Errno 2] No such file or directory: '../data/script/western/clean/'


,genre,nb_of_movies,nb_male,nb_female,nb_male_occurences,nb_female_occurences,nb_prota_male,nb_prota_female
0,Action,87,1618,553,41836,14221,833,324
1,Adventure,59,1071,400,31179,10628,567,236
2,Animation,15,213,112,6948,3071,120,65
3,Biography,18,423,168,10013,3884,217,88
4,Comedy,71,991,717,32951,24311,518,405
5,Crime,58,1044,468,29413,11843,556,254
6,Drama,110,1596,1065,46927,36255,800,589
7,Family,1,3,19,61,655,1,11
8,Fantasy,14,220,118,6071,4329,110,74
9,History,4,67,42,1319,1592,38,23
